# Homework 7 (30 marks)
Create a copy of the notebook to start answering the questions. Name your notebook in the format HW7_lastname_firstname.ipynb to facilitate the grading process.

Answer all the questions, test your code to ensure there are no errors and the results are as expected. Once you have answered all the questions, save the final copy, then go to File-> click on Download.ipynb. Once the local copy has been downloaded, submit your file on Blackboard under the corresponding assignment section. Also provide us a link to your notebook during submission.

NOTE: Please give the TAs the permission to access your notebooks through the links you have provided during submission.

The due date of this homework is 04/23/2021 (Friday).

Please ensure you follow all the steps mentioned in the homework.

You can submit your solutions any number of times until the deadline.

The datasets used in this homework can be found in the google drive link below -

https://drive.google.com/drive/folders/1NxCh4X7u7wVo5aHojxjLNs9wC7B7zJhb?usp=sharing

Follow the necessary steps to import data to test your code. You can use any method to read your data in the notebook. We will not be grading the methods you use. We will only grade the code from the point where you read the dataset into a pandas dataframe - (pd.read_csv('file_name'))

Import all the libraries you require in the cell below.


In [ ]:
# Enter your code here
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

## Implement the Ensemble methods learnt in class and compare their accuarcies.

The dataset you are going to be using for homework is the **Wisconsin Breast Cancer dataset (cancer.csv)**

The dataset contains a total number of 10 features labeled in either benign or malignant classes. The features have 699 instances out of which 16 feature values are missing. The dataset only contains numeric values.

Attribute Information:

1. Sample code number: id number
2. Clump Thickness: 1 - 10
3. Uniformity of Cell Size: 1 - 10
4. Uniformity of Cell Shape: 1 - 10
5. Marginal Adhesion: 1 - 10
6. Single Epithelial Cell Size: 1 - 10
7. Bare Nuclei: 1 - 10
8. Bland Chromatin: 1 - 10
9. Normal Nucleoli: 1 - 10
10. Mitoses: 1 - 10
11. Class: (2 for benign, 4 for malignant) (**target variable**)

For more information: https://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+(original)

### 1. Read the dataset into variable called '**data**' (1 mark)

In [ ]:
pd.set_option('display.max_columns', 100)

# Enter your code here
data = pd.read_csv('cancer.csv')

### **Preprocessing**: Data needs to be preprocessed before implementing ensemble methods. It is done for you here. 
### Run the below code first and then answer the questions from 2 - 7.

#### Deleting unnecessary columns: The column "Sample code number" is just an indicator and it's of no use in the modeling. So, let's drop it:


In [ ]:
data.drop(['Sample Code Number'],axis = 1, inplace = True)

#### Handling missing values : 
As mentioned earlier, the dataset contains missing values. The column named "Bland Chromatin" contains them. The missing values are represneted as "?". 

Replace those "?"s with 0's and impute them with Mean Imputation

In [ ]:
data['Bland Chromatin']

0       1
1      10
2       2
3       4
4       1
       ..
694     2
695     1
696     3
697     4
698     5
Name: Bland Chromatin, Length: 699, dtype: object

In [ ]:
data.replace('?',0, inplace=True)

In [ ]:
# Convert the DataFrame object into NumPy array otherwise you will not be able to impute
values = data.values
# Now impute it
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputedData = imputer.fit_transform(values)

#### Normalizing the data:
Ranges of the features of the dataset are not the same. This may cause a problem. A small change in a feature might not affect the other. To address this problem, normalize the ranges of the features to a uniform range, in this case, 0 - 1.

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
normalizedData = scaler.fit_transform(imputedData)
cols = ['Clump Thickness', 'Uniformity of Cell Size', 'Uniformity of Cell Shape', 'Marginal Adhesion', 'Single Epithelial Cell Size', 'Bland Chromatin', 'Bare Nuclei', 'Normal Nucleoli', 'Mitosis','Class']
normalizedData = pd.DataFrame(normalizedData, columns=cols)
# print(normalizedData.head())
# print(normalizedData)
normalizedData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Clump Thickness              699 non-null    float64
 1   Uniformity of Cell Size      699 non-null    float64
 2   Uniformity of Cell Shape     699 non-null    float64
 3   Marginal Adhesion            699 non-null    float64
 4   Single Epithelial Cell Size  699 non-null    float64
 5   Bland Chromatin              699 non-null    float64
 6   Bare Nuclei                  699 non-null    float64
 7   Normal Nucleoli              699 non-null    float64
 8   Mitosis                      699 non-null    float64
 9   Class                        699 non-null    float64
dtypes: float64(10)
memory usage: 54.7 KB


### Data preprocessing is done and now you will answer the below questions using the **normalizedData**: 

### 2. Split the data into test and training data with test size - 30%. Compute the baseline classification accuracy for X_train. (3 marks)

In [ ]:
# Enter your code here
X = normalizedData.iloc[:, :-1]
y = normalizedData.iloc[:, -1]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

from sklearn.dummy import DummyClassifier
dummy = DummyClassifier(strategy = 'most_frequent')
dummy.fit(X_train, y_train)
accuracy = dummy.score(X_test, y_test)

print("baseline accuracy:", accuracy)

baseline accuracy: 0.6476190476190476


### 3.  Bagging : Build a generic Bagging ensemble and print the accuracy (4 marks)
---


Hyperparameters:

Base estimator = DecisionTreeClassifier

n_estimators = 10

random_state = 42

---


In [ ]:
# Generic Bagging model
# Enter your code here
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

bagging_model = BaggingClassifier(base_estimator = DecisionTreeClassifier(), n_estimators = 10, random_state = 42)
bagging_model.fit(X_train, y_train)
bagging_predictions = bagging_model.predict(X_test)
bagging_accuracy = accuracy_score(y_test, bagging_predictions)

print('bagging ensemble accuracy:', bagging_accuracy)

bagging ensemble accuracy: 0.9952380952380953


### 4. RandomForest : (7 marks)
#### a) Build a Random Forest model and print the accuracy (4 marks)
---

Constructor arguments: 


n_estimators = 100, max_features = 7 and random_state = 42 


---




In [ ]:
# Random Forest model
# Enter your code here
from sklearn.ensemble import RandomForestClassifier

random_forest_model = RandomForestClassifier(n_estimators = 100, max_features = 7, random_state = 42)
random_forest_model.fit(X_train, y_train)
random_forest_predictions = random_forest_model.predict(X_test)
random_forest_accuracy = accuracy_score(y_test, random_forest_predictions)

print('random forest model accuracy:', random_forest_accuracy)

random forest model accuracy: 0.9904761904761905


####  b) Calculate the top 3 important features for the above **RandomForest** model and print them (3 marks)

In [ ]:
# Top 3 features for RandomForest
# Enter your code here

feature_importances_rf = random_forest_model.feature_importances_
features_rf = X_train.columns
feature_importances_rf_df = pd.DataFrame({'features': features_rf, 'feature importances': feature_importances_rf})
feature_importances_rf_df.sort_values('feature importances', ascending = False, inplace = True)

print('top three most important features:')
print(feature_importances_rf_df.head(3))

top three most important features:
                   features  feature importances
1   Uniformity of Cell Size             0.407630
5           Bland Chromatin             0.300677
2  Uniformity of Cell Shape             0.122144


### 5. Boosting: (7 marks)
#### a) Build an AdaBoost model with training data and print the accuracy (4 marks)
---

Hyperparameters:

Base estimator = DecisionTreeClassifier, max_depth = 4

n_estimators = 200

random_state = 42

learning_rate = 0.05


---









In [ ]:
# AdaBoost Classification
# Enter your code here
from sklearn.ensemble import AdaBoostClassifier

base_estimator = DecisionTreeClassifier(max_depth = 4)
adaboost_model = AdaBoostClassifier(base_estimator = base_estimator, n_estimators = 200, random_state = 42, learning_rate = 0.05)
adaboost_model.fit(X_train, y_train)
adaboost_predictions = adaboost_model.predict(X_test)
adaboost_accuracy = accuracy_score(y_test, adaboost_predictions)

print('adaboost model accuracy:', adaboost_accuracy)

adaboost model accuracy: 0.9809523809523809


#### b) Calculate the top 3 important features for the above **AdaBoost** model and print them (3 marks)

In [ ]:
# Top 3 features for AdaBoost
# Enter your code here

feature_importances_adaboost = adaboost_model.feature_importances_
features_adaboost = X_train.columns
feature_importances_adaboost_df = pd.DataFrame({'features': features_adaboost, 'feature importances': feature_importances_adaboost})
feature_importances_adaboost_df.sort_values('feature importances', ascending = False, inplace = True)

print('top three most important features')
print(feature_importances_adaboost_df.head(3))

top three most important features
                   features  feature importances
0           Clump Thickness             0.322583
2  Uniformity of Cell Shape             0.261773
5           Bland Chromatin             0.215702


### 6. Voting : Using a voting classifier, build an ensemble of RandomForestClassifier, DecisionTreeClassifier, Support Vector Machine and Logistic Regression. (7 marks)


---


Use max_depth = 4, n_estimators = 200, voting = soft

In [ ]:
# Voting Ensemble for Classification
# Enter your code here
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

random_forest = RandomForestClassifier(n_estimators = 200, max_depth = 4)
decision_tree = DecisionTreeClassifier(max_depth = 4)
support_vector_machine = SVC(probability = True)
logistic_regression = LogisticRegression()

voting_classifier = VotingClassifier(estimators = [('rf', random_forest), ('dt', decision_tree), ('svm', support_vector_machine), ('log', logistic_regression)], voting = 'soft')
voting_classifier.fit(X_train, y_train)
voting_predictions = voting_classifier.predict(X_test)
voting_accuracy = accuracy_score(y_test, voting_predictions)

print('voting classifier accuracy:', voting_accuracy)

voting classifier accuracy: 0.9857142857142858


### 7. Mention the best model among the above 4 models and its accuracy (1 mark)

In [ ]:
# Write your answer here
print("baseline accuracy:", accuracy)
print('bagging ensemble accuracy:', bagging_accuracy)
print('random forest model accuracy:', random_forest_accuracy)
print('adaboost model accuracy:', adaboost_accuracy)
print('voting classifier accuracy:', voting_accuracy)

print('\nthe best model would be the bagging ensemble with an accuracy of', bagging_accuracy)

baseline accuracy: 0.6476190476190476
bagging ensemble accuracy: 0.9952380952380953
random forest model accuracy: 0.9904761904761905
adaboost model accuracy: 0.9809523809523809
voting classifier accuracy: 0.9857142857142858

the best model would be the bagging ensemble with an accuracy of 0.9952380952380953
